Задача за DSCM006
=================

Предаваме като .ipynb в Moodle в курса DSCM006 до 15.02.2022 г.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [3]:
df_train = pd.read_csv('X_train.csv', encoding='"ISO-8859-1"')
df_test = pd.read_csv('X_test.csv', encoding='"ISO-8859-1"')

1.   Направете основен експлораторен анализ на обучителния набор;
2.   Намалете размерността на данните (PCA, correlation, VIF);
3.   Направете необходимите операции, за да присвоите на категорийните променливи числови стойности, така че те да могат да бъдат използвани в модел;
4.   Мащабирайте данните - необходимо е най-вече за SVM от моделите, които ще използвате;
5.   Използвайки кръстосана валидация с обучителния набор, намерете най-добрия модел, варирайки хиперпараметрите. Използвайте логистична регресия и метода на опорните вектори. Опитайте всички възможности за тези модели и чрез GridSearchCV или по друг начин намерете най-добрия модел като експериментирате с различни приложими метрики за оценка на точността;
6.   За SVM, scikit-learn предлага ядрата ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, които в т.5 се очаква да използвате в решението си. Освен тях, опитайте модел със сплайн ядро, което е описано [в доклад на Steve Gunn от 1998 г](http://www.svms.org/tutorials/Gunn1998.pdf). За начина на внедряване на код за ядра в sklearn можете да ползвате указанията по [тази](https://krypticmouse.hashnode.dev/training-svm-over-custom-kernels) връзка, а за реализация на сплайн ядро може да използвате [този](https://github.com/gmum/pykernels/blob/master/pykernels/regular.py) код. Ако намерите по-подходящи ядра, опитайте и с тях;
7.   *По желание.* Проучете как работи XGBoost и опитайте да решите задачата с него;
8.   След като сте избрали най-добрия модел, вижте как той се справя на тестовия набор. 

In [6]:
df_train["Bad_Flag"].value_counts(dropna=False)

0    5841
1     750
Name: Bad_Flag, dtype: int64